In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from torchvision.transforms.functional import to_pil_image
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
csv_path = "C:/Users/HP/Downloads/GLEASON/Train_with_Gleason_and_Labels.csv"
df = pd.read_csv(csv_path)
df['risk_class'] = df['risk_label'].map({
    0: 0,  # Gleason 0 → Low Risk
    1: 0,  # Gleason 6 → Low Risk
    2: 1,  # Gleason 7 → Intermediate Risk
    3: 2   # Gleason 8–10 → High Risk
})
df['image_path'] = df['image_path'].apply(lambda x: os.path.join("C:/Users/HP/Downloads/GLEASON/Train Imgs", os.path.basename(x)))
df['mask_path'] = df['mask_path'].apply(lambda x: os.path.join("C:/Users/HP/Downloads/GLEASON", x.strip('./')))

print("Loaded:", len(df), "rows")
print("Example:", df.iloc[0])

Loaded: 1171 rows
Example: image_path         C:/Users/HP/Downloads/GLEASON/Train Imgs\slide...
mask_path          C:/Users/HP/Downloads/GLEASON\Maps1_T/slide001...
expert_id                                                          1
primary_grade                                                      4
secondary_grade                                                    4
gleason_score                                                      8
risk_label                                                         3
risk_class                                                         2
Name: 0, dtype: object


In [8]:
class GleasonGradeDataset(Dataset):
    def __init__(self, df, target="primary"):
        self.df = df.reset_index(drop=True)
        self.target = target
        self.grade_map = {3: 0, 4: 1, 5: 2}
        self.transform_img = T.Compose([
            T.Resize((512, 512)),
            T.ToTensor(),
            T.Normalize([0.5]*3, [0.5]*3)
        ])
        self.transform_mask = T.Compose([
            T.Resize((512, 512)),
            T.ToTensor()
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['image_path']).convert("RGB")
        mask = Image.open(row['mask_path']).convert("L")

        image_tensor = self.transform_img(image)
        mask_tensor = self.transform_mask(mask)
        input_tensor = torch.cat([image_tensor, mask_tensor], dim=0)  # 4-channel input

        mask_np = np.array(mask)
        valid = [3, 4, 5]
        grade_pixels = {v: np.sum(mask_np == v) for v in valid}
        sorted_grades = sorted(grade_pixels.items(), key=lambda x: x[1], reverse=True)
        primary = sorted_grades[0][0] if sorted_grades else 3
        secondary = sorted_grades[1][0] if len(sorted_grades) > 1 else primary
        label = primary if self.target == "primary" else secondary

        return input_tensor, self.grade_map.get(label, 0)

In [9]:
class CNNGradePredictor(nn.Module):
    def __init__(self):
        super(CNNGradePredictor, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(4, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),  # 256x256
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),  # 128x128
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),  # 64x64
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2),  # 32x32
            nn.Conv2d(256, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(), nn.AdaptiveAvgPool2d((1, 1))  # 1x1
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 128), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(128, 3)  # 3 classes: grade 3, 4, 5
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

In [10]:
def train_model(model, train_loader, val_loader, class_weights, save_path, label_name=""):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    best_f1 = 0
    patience, trigger = 5, 0

    for epoch in range(1, 12):
        model.train()
        total_loss, preds, labels = 0, [], []

        for x, y in tqdm(train_loader, desc=f"Epoch {epoch} - Training"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            preds.extend(torch.argmax(out, 1).cpu().numpy())
            labels.extend(y.cpu().numpy())

        train_acc = accuracy_score(labels, preds)
        train_f1 = f1_score(labels, preds, average='macro')

        model.eval()
        val_preds, val_labels = [], []

        with torch.no_grad():
            for x, y in tqdm(val_loader, desc=f"Epoch {epoch} - Validation"):
                x, y = x.to(device), y.to(device)
                out = model(x)
                pred = torch.argmax(out, dim=1)
                val_preds.extend(pred.cpu().numpy())
                val_labels.extend(y.cpu().numpy())

        val_acc = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='macro')
        cm = confusion_matrix(val_labels, val_preds)

        print(f"\nEpoch {epoch} Results:")
        print(f"Train Loss: {total_loss:.4f} | Train Acc: {train_acc:.4f} | Train F1: {train_f1:.4f}")
        print(f"Val   Acc: {val_acc:.4f} | Val   F1: {val_f1:.4f}")
        print("Confusion Matrix:\n", cm)

        if val_f1 > best_f1:
            best_f1 = val_f1
            trigger = 0
            torch.save(model.state_dict(), save_path)
            print("Best model saved.")
        else:
            trigger += 1
            print(f"No improvement. Patience: {trigger}/{patience}")
            if trigger >= patience:
                print("Early stopping triggered.")
                break

In [11]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['risk_class'], random_state=42)

train_primary = GleasonGradeDataset(train_df, target="primary")
val_primary = GleasonGradeDataset(val_df, target="primary")
train_secondary = GleasonGradeDataset(train_df, target="secondary")
val_secondary = GleasonGradeDataset(val_df, target="secondary")

train_loader_primary = DataLoader(train_primary, batch_size=8, shuffle=True)
val_loader_primary = DataLoader(val_primary, batch_size=8)
train_loader_secondary = DataLoader(train_secondary, batch_size=8, shuffle=True)
val_loader_secondary = DataLoader(val_secondary, batch_size=8)

y_primary = [train_primary[i][1] for i in tqdm(range(len(train_primary)))]
y_secondary = [train_secondary[i][1] for i in tqdm(range(len(train_secondary)))]

class_weights_primary = compute_class_weight(class_weight="balanced", classes=np.array([0, 1, 2]), y=y_primary)
class_weights_secondary = compute_class_weight(class_weight="balanced", classes=np.array([0, 1, 2]), y=y_secondary)

class_weights_primary_tensor = torch.tensor(class_weights_primary, dtype=torch.float32).to(device)
class_weights_secondary_tensor = torch.tensor(class_weights_secondary, dtype=torch.float32).to(device)

  0%|          | 0/936 [00:00<?, ?it/s]

  0%|          | 0/936 [00:00<?, ?it/s]

In [12]:
model_primary = CNNGradePredictor().to(device)
train_model(
    model=model_primary,
    train_loader=train_loader_primary,
    val_loader=val_loader_primary,
    class_weights=class_weights_primary_tensor,
    save_path="cnn_primary_model4.pth",
    label_name="Primary"
)

model_secondary = CNNGradePredictor().to(device)
train_model(
    model=model_secondary,
    train_loader=train_loader_secondary,
    val_loader=val_loader_secondary,
    class_weights=class_weights_secondary_tensor,
    save_path="cnn_secondary_model4.pth",
    label_name="Secondary"
)

Epoch 1 - Training:   0%|          | 0/117 [00:00<?, ?it/s]

Epoch 1 - Validation:   0%|          | 0/30 [00:00<?, ?it/s]


Epoch 1 Results:
Train Loss: 131.4047 | Train Acc: 0.5032 | Train F1: 0.3642
Val   Acc: 0.4936 | Val   F1: 0.3995
Confusion Matrix:
 [[38 46 21]
 [20 74 28]
 [ 1  3  4]]
Best model saved.


Epoch 2 - Training:   0%|          | 0/117 [00:00<?, ?it/s]

Epoch 2 - Validation:   0%|          | 0/30 [00:00<?, ?it/s]

...

Epoch 10 - Validation:   0%|          | 0/30 [00:00<?, ?it/s]


Epoch 10 Results:
Train Loss: 110.3805 | Train Acc: 0.6966 | Train F1: 0.4721
Val   Acc: 0.6638 | Val   F1: 0.4366
Confusion Matrix:
 [[103  21   0]
 [ 53  53   0]
 [  2   3   0]]
No improvement. Patience: 2/5


Epoch 11 - Training:   0%|          | 0/117 [00:00<?, ?it/s]

Epoch 11 - Validation:   0%|          | 0/30 [00:00<?, ?it/s]


Epoch 11 Results:
Train Loss: 112.2556 | Train Acc: 0.7147 | Train F1: 0.4844
Val   Acc: 0.7319 | Val   F1: 0.4907
Confusion Matrix:
 [[ 72  52   0]
 [  6 100   0]
 [  2   3   0]]
Best model saved.


In [13]:
model_primary.load_state_dict(torch.load("cnn_primary_model4.pth"))
model_secondary.load_state_dict(torch.load("cnn_secondary_model4.pth"))

model_primary.eval()
model_secondary.eval()

CNNGradePredictor(
  (features): Sequential(
    (0): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [14]:
class CNN4Dataset(Dataset):
    def __init__(self, df, cnn1_model, cnn2_model, transform=None):
        self.df = df.reset_index(drop=True)
        self.cnn1 = cnn1_model.eval()
        self.cnn2 = cnn2_model.eval()
        self.transform_img = transform or T.Compose([
            T.Resize((512, 512)),
            T.ToTensor(),
            T.Normalize([0.5]*3, [0.5]*3)
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['image_path']
        mask_path = row['mask_path']

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")
        img_tensor = self.transform_img(image)

        # Dummy mask needed to form 4-channel input to CNN1/2
        mask_tensor = T.Resize((512, 512))(T.ToTensor()(mask))
        img4_tensor = torch.cat([img_tensor, mask_tensor], dim=0).unsqueeze(0).to(device)

        with torch.no_grad():
            pred_primary = self.cnn1(img4_tensor)
            pred_secondary = self.cnn2(img4_tensor)

        primary_grade = torch.argmax(pred_primary, dim=1).item() + 3
        secondary_grade = torch.argmax(pred_secondary, dim=1).item() + 3
        gleason_score = primary_grade + secondary_grade

        # Risk class logic: 0 = 0/6, 1 = 7, 2 = 8/9/10
        if gleason_score in [0, 6]:
            risk = 0
        elif gleason_score == 7:
            risk = 1
        elif gleason_score in [8, 9, 10]:
            risk = 2
        else:
            risk = 0

        # Construct 6-channel input: RGB + pri + sec + score
        input_tensor = torch.cat([
            img_tensor,
            torch.tensor([[primary_grade / 5.0]]).repeat(1, 512, 512),
            torch.tensor([[secondary_grade / 5.0]]).repeat(1, 512, 512),
            torch.tensor([[gleason_score / 10.0]]).repeat(1, 512, 512)
        ], dim=0)

        return input_tensor, torch.tensor(risk, dtype=torch.long)

In [15]:
cnn4_train_dataset = CNN4Dataset(train_df, model_primary, model_secondary)
cnn4_val_dataset = CNN4Dataset(val_df, model_primary, model_secondary)

cnn4_train_loader = DataLoader(cnn4_train_dataset, batch_size=8, shuffle=True)
cnn4_val_loader = DataLoader(cnn4_val_dataset, batch_size=8, shuffle=False)

# Check risk label distribution for weights
risk_labels_train = [cnn4_train_dataset[i][1].item() for i in tqdm(range(len(cnn4_train_dataset)))]
print("✅ Risk class distribution:", pd.Series(risk_labels_train).value_counts())

# Compute class weights
class_weights_cnn4 = compute_class_weight(class_weight='balanced', classes=np.array([0,1,2]), y=risk_labels_train)
class_weights_cnn4_tensor = torch.tensor(class_weights_cnn4, dtype=torch.float32).to(device)

  0%|          | 0/936 [00:00<?, ?it/s]

✅ Risk class distribution: 1    666
2    183
0     87
Name: count, dtype: int64


In [16]:
class CNN4RiskClassifier(nn.Module):
    def __init__(self):
        super(CNN4RiskClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(6, 64, kernel_size=3, padding=1),  # Input: 6-channel image
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),  # [B, 512]
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 3)  # Final 3-class risk output
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [17]:
cnn4_model = CNN4RiskClassifier().to(device)
loss_fn = nn.CrossEntropyLoss(weight=class_weights_cnn4_tensor)
optimizer = torch.optim.Adam(cnn4_model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

In [18]:
epochs = 20
patience = 5
best_f1 = 0
early_stop = 0

for epoch in range(1, epochs + 1):
    cnn4_model.train()
    train_loss, all_preds, all_labels = 0, [], []

    for inputs, targets in tqdm(cnn4_train_loader, desc=f"Epoch {epoch} - Training"):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = cnn4_model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        all_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        all_labels.extend(targets.cpu().numpy())

    train_acc = accuracy_score(all_labels, all_preds)
    train_f1 = f1_score(all_labels, all_preds, average='macro')

    # === Validation ===
    cnn4_model.eval()
    val_preds, val_labels = [], []

    with torch.no_grad():
        for inputs, targets in tqdm(cnn4_val_loader, desc=f"Epoch {epoch} - Validation"):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = cnn4_model(inputs)
            preds = torch.argmax(outputs, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(targets.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    val_f1 = f1_score(val_labels, val_preds, average='macro')
    cm = confusion_matrix(val_labels, val_preds)

    print(f"\n📊 Epoch {epoch} Results:")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Train F1: {train_f1:.4f}")
    print(f"Val   Acc: {val_acc:.4f} | Val   F1: {val_f1:.4f}")
    print("Confusion Matrix:\n", cm)

    scheduler.step(val_f1)

    if val_f1 > best_f1:
        best_f1 = val_f1
        early_stop = 0
        torch.save(cnn4_model.state_dict(), "Fused_Final_cnn4.pth")
        print("✅ Best model saved.")
    else:
        early_stop += 1
        print(f"No improvement. Patience: {early_stop}/{patience}")
        if early_stop >= patience:
            print("⏹️ Early stopping triggered.")
            break

Epoch 1 - Training:   0%|          | 0/117 [00:00<?, ?it/s]

Epoch 1 - Validation:   0%|          | 0/30 [00:00<?, ?it/s]


📊 Epoch 1 Results:
Train Loss: 39.2999 | Train Acc: 0.8974 | Train F1: 0.8468
Val   Acc: 0.9915 | Val   F1: 0.9891
Confusion Matrix:
 [[ 21   0   0]
 [  0 176   2]
 [  0   0  36]]
✅ Best model saved.


Epoch 2 - Training:   0%|          | 0/117 [00:00<?, ?it/s]

Epoch 2 - Validation:   0%|          | 0/30 [00:00<?, ?it/s]

...

Epoch 4 - Training:   0%|          | 0/117 [00:00<?, ?it/s]

Epoch 4 - Validation:   0%|          | 0/30 [00:00<?, ?it/s]


📊 Epoch 4 Results:
Train Loss: 10.7937 | Train Acc: 0.9701 | Train F1: 0.9517
Val   Acc: 1.0000 | Val   F1: 1.0000
Confusion Matrix:
 [[ 21   0   0]
 [  0 178   0]
 [  0   0  36]]
✅ Best model saved.


Epoch 5 - Training:   0%|          | 0/117 [00:00<?, ?it/s]

KeyboardInterrupt: 